## Hypothesis Testing
### Aim : To Prove University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. 

In [1]:
#importing important libraries
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [2]:
def get_list_of_university_towns():
    
    fhand = open('university_towns.txt')
    #dict_=dict()
    lst=list()
    for lines in fhand:
        lines=lines.rstrip()
        if '[ed' in lines:
            state=lines.split('[')[0]    
        else:
            region=lines.split(' (')[0]
            x=[state,region]
            lst.append(x)
    df=pd.DataFrame(lst,columns=["State", "RegionName"])
    return df
get_list_of_university_towns()

FileNotFoundError: [Errno 2] No such file or directory: 'university_towns.txt'

In [3]:
def get_recession_start():

    dx=pd.read_excel('gdplev.xls',skiprows=7)
    df=dx[['Unnamed: 4','Unnamed: 6']] #selecting only wanted columns
    df=df[212:] # only looking for years after 2000
    df.rename(columns={'Unnamed: 4':'yq','Unnamed: 6':'chained dollars'},inplace=True)
    series=df['chained dollars']
    lst=list()
    for x in series:
        lst.append(x)
    i=0
    while i<64:
        if (lst[i]>lst[i+1]) & lst[i+1]>lst[i+2]:    #Finding recession start year using while loop 
                break
            
        i=i+1
    x=df.iloc[i+1]
    
    #for x in df.
    return x['yq']
get_recession_start()

FileNotFoundError: [Errno 2] No such file or directory: 'gdplev.xls'

In [ ]:
def get_recession_end():

    recession=get_recession_start()
    dx=pd.read_excel('gdplev.xls',skiprows=7)
    df=dx[['Unnamed: 4','Unnamed: 6']]
    #df=df[212:]
    df.rename(columns={'Unnamed: 4':'yq','Unnamed: 6':'chained dollars'},inplace=True)
    for x in df.index:
        if df.loc[x]['yq']==recession:
            break
    df=df[x:]
    series=df['chained dollars']
    lst=list()
    for x in series:
        lst.append(x)
    i=2
    while i<(len(lst)):
        if (lst[i]>lst[i-1]) & (lst[i-1]>lst[i-2]):
                break
            
        i=i+1
    x=df.iloc[i]
    
    return x['yq']
get_recession_end()

In [ ]:
def get_recession_bottom():
    
    import pandas as pd
    import numpy as np
    recession_start=get_recession_start()
    recession_end=get_recession_end()
    dx=pd.read_excel('gdplev.xls',skiprows=7)
    df=dx[['Unnamed: 4','Unnamed: 6']]
    #df=df[212:]
    df.rename(columns={'Unnamed: 4':'yq','Unnamed: 6':'chained dollars'},inplace=True)
    for x in df.index:
        if df.loc[x]['yq']==recession_start:
            break
    for y in df.index:
        if df.loc[y]['yq']==recession_end:
            break
    df=df[x:y]
    df1=df['chained dollars']
    x=np.argmin(df1)
    z=df.loc[x]['yq']
    return z
get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
 
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    import pandas as pd
    import numpy as np
    df=pd.read_csv('City_Zhvi_AllHomes.csv')
    columns=list()
    new_columns=list()
    
    #for eliminating years before 2000
    
    for x in df.columns:
        y=x.split('-')
        try:
            y1=float(y[0])
            if y1>=2000:
                columns.append(x)
        except:
            columns.append(x)

    df=df[columns]
    
    df.replace(states,inplace=True) #replacing state shortcut names
    df=df.set_index(["State","RegionName"])
    new_columns=columns[6:]
    #new_columns=pd.to_datetime(new_columns)
    df=df[new_columns]
    
    #converting to datetime in order to join them quarterly
    df.columns= pd.to_datetime(df.columns)
    df=df.resample('Q',axis=1).mean()
    d=dict()
    
    #renaming the datetime into required format i.e , 2000 q1 etc
    
    for x in df.columns:
        y=str(x)
        z=y.split('-')
        if float(z[1])/3==1:
            a=z[0]+'q1'
            d[x]=a
        elif float(z[1])/3==2:
            a=z[0]+'q2'
            d[x]=a
        elif float(z[1])/3==3:
            a=z[0]+'q3'
            d[x]=a
        elif float(z[1])/3==4:
            a=z[0]+'q4'
            d[x]=a
    df.rename(columns=d,inplace=True)
    
    return df
convert_housing_data_to_quarters()


In [ ]:
def run_ttest():
    
    import pandas as pd
    from scipy import stats 
    unv_town=get_list_of_university_towns()
    home_rent=convert_housing_data_to_quarters()
    group1=pd.merge(unv_town,home_rent,how='inner',left_on=['State','RegionName'],right_index=True)
    #main=pd.merge(unv_town,home_rent,how='outer',left_on=['State','RegionName'],right_index=True,indicator='_flag')
    main=pd.merge(home_rent.reset_index(),unv_town,on=unv_town.columns.tolist(),indicator='_flag',how='left')
    group2=main[main['_flag']!='both']
    rec_start=get_recession_start()
    rec_bottom=get_recession_bottom()
    i=0
    while True:
        if home_rent.columns[i]==rec_start:
            a=home_rent.columns[i-1]
            break
        i=i+1
    group1['price_ratio']=group1[a].div(group1[rec_bottom])
    group2['price_ratio']=group2[a].div(group2[rec_bottom])
    x=stats.ttest_ind(group1['price_ratio'],group2['price_ratio'],nan_policy='omit').pvalue
    if x<0.1:
        different=True
    else:
        different=False
    if np.mean(group1['price_ratio'])<np.mean(group2['price_ratio']):
        better="university town"
    else:
        better="non-university town"
    p=x
    return (different,p,better)
    
run_ttest()